In [1]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_classes
from raceDB import build_dataset, build_pred_dataset
import importlib
import datetime
from training_testing import validate_model, train_regular, train_log, train_super_batch, train_super_batch_KL, train_super_batch_L1, train_regular_L1,train_regular_one_hot, train_double_loss_regular
from model_saver import model_saver, model_saver_wandb
import training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

In [4]:
def custom_l2(output, target):
    loss = torch.mean(abs(output-target), dim=1)
    return loss

In [5]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [6]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
hidden_size = 64
raceDB = build_dataset('gru_inputs_new_v_basic.npy', hidden_size ,state_filter=["NSW"], margin_type='raw')
raceDB.create_new_weights_v2()
#raceDB.adjust_weights({"Dapto":10, "Gunnedah":10})

(1177985, 19)
(1177985, 19)
(290988, 22)
Latest date = 2023-03-15 00:00:00
num_features_per_dog=13


  0%|          | 0/14749 [00:00<?, ?it/s]

c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


  0%|          | 0/40842 [00:00<?, ?it/s]

c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


number of races = 40842, number of unique dogs = 14749
0          (Dubbo, 400.0)
1          (Dubbo, 318.0)
2          (Dubbo, 318.0)
3        (Grafton, 305.0)
4        (Grafton, 305.0)
               ...       
40837      (Taree, 314.0)
40838      (Taree, 392.0)
40839      (Taree, 537.0)
40840      (Taree, 392.0)
40841      (Taree, 392.0)
Length: 40842, dtype: object


In [7]:
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
raceDB.create_test_split_date(date)

Train examples 32452, Test examples 8390


In [8]:
raceDB.create_dogs_test_split_date()

  0%|          | 0/14749 [00:00<?, ?it/s]

In [9]:
raceDB.attach_races_to_dog_input()

  0%|          | 0/14749 [00:00<?, ?it/s]

In [13]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="NEW GRU", config=config):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss", summary="min")
      wandb.define_metric("test_accuracy", summary="max")
      wandb.define_metric("bfprofit", summary="max")
      wandb.define_metric("multibet profit", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy

      model = rnn_classes.GRUNet(187,config['hidden_size'])


      raceDB.to_cuda()

      criterion = nn.SmoothL1Loss(reduction='mean', beta=5)
      # optimizer = optim.RMSprop(model.parameters(), lr=config['learning_rate'])
      optimizer = optim.Adadelta(model.parameters())

      model = model.to(device)
      # optimizer = optimizer.to(device)
      print(model)

      # and use them to train the model
      try:
        # train_double_loss_regular(model, dataset, criterion, optimizer, scheduler, config, crit2=custom_l2)
        training_testing_gru.train_regular(model, dataset, criterion, optimizer, 'na', config)
      except KeyboardInterrupt:
        print("finished Early")
        
      dataset.create_hidden_states_dict()
      model_saver_wandb(model, optimizer, 450, 0.1, dataset.hidden_states_dict_gru, model_name="long nsw new  22000 RUN")
      if sweep:
        raceDB.reset_all_lstm_states
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset, optimizer)

In [15]:
len(raceDB.raceIDs)
wandb_config_static = {'hidden_size':hidden_size,'batch_size': 750, 'dropout': 0.3, 'epochs': 10_000, 'f1_layer_size': 256, 'f2_layer_size': 64 , 'learning_rate': 0.000087, 'loss': 'L1', 'l1_beta':0.1,  'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1,'batch_before_backwards':7}

In [17]:
#(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static)
importlib.reload(training_testing_gru)
importlib.reload(rnn_classes)
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

{'hidden_size': 64, 'batch_size': 750, 'dropout': 0.3, 'epochs': 10000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 8.7e-05, 'loss': 'L1', 'l1_beta': 0.1, 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1, 'batch_before_backwards': 7}
10000
{'hidden_size': 64, 'batch_size': 750, 'dropout': 0.3, 'epochs': 10000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 8.7e-05, 'loss': 'L1', 'l1_beta': 0.1, 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1, 'batch_before_backwards': 7}
GRUNet(
  (gru): GRU(187, 64)
  (relu0): ReLU()
  (fc0): Linear(in_features=64, out_features=1, bias=True)
  (output_fn): Identity()
)


  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

...

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

finished Early
created path



ROI < 30,▄▄▂▂▅▄▃▄▁▃▃▄▂▅▃▂▅▅▄▄▆▆▅▄█▅█▅▃▇▃▆▅▄▅▅▄▂▂▆
accuracy,▁▃▃▄▄▅▅▅▄▅▆▅▆▆▆▆▇▇▆▆█▆▇▇█▇█▇▇▇▇█▇▇▇▇▇▇▇▇
batch_loss,▅▆▃▆▄▄█▃▆▂▄█▃▂▃▃▃▂▂▂▃▂▄▂▂▆▃▄▂▂▆▂▂▁▂▂▁▂▁▁
multibet outlay < 30,▁▂▄▄▄▆▆▆▆▆▇▇▆▇▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█▇▇▇██▇█
multibet profit,▁▂▁▁▄▆▃▄▂▄▆▅▂▆▆▃▅▇▃▃▅▇▇▄▇▆▇▅▃█▃▇▅▃▅▅▅▂▁█
multibet profit < 30,▃▃▂▂▄▄▃▃▁▂▃▄▂▅▃▂▅▄▄▃▆▅▅▄█▅█▅▃█▃▆▅▄▅▅▄▂▂▆
multibet profit < 30 sd,▁▂▄▅▅▇▆▆▆▆▇█▆▇▇▅▆▆▅▆▆█▇▇▇▆▇▇▆█▆▇▇▇▇█▇▇▆█
ROI < 30,0.09759
accuracy,0.21257
batch_loss,2.17327
multibet outlay < 30,204247.03259


: 

In [ ]:
len_test = len(raceDB.test_dog_ids)
test_idx = range(0,len_test)

# dog_inputs = [[z.full_input for z in inner] for inner in [x for x in raceDB.train_dogs.values()]]
dogs = [x for x in  raceDB.test_dogs.values()]  #[Dog]
dog_input = [inner for inner in [x for x in raceDB.get_dog_test(test_idx)]] #[[DogInput]]

for dog in dogs:
    dog.hidden = dog.hidden.to('cuda:0')

train = [torch.stack(n,0) for n in [[z.full_input for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]]
X = pack_sequence(train, enforce_sorted=False)

dogs_sorted = [dogs[x] for x in X[3]]
hidden_in = torch.unsqueeze(torch.stack([x.hidden for x in dogs_sorted]).to('cuda:0'),0)

In [ ]:
output = model(X,h=hidden_in, train=False)

23
20
8
22
9
4
18
2
8
4
8
1
10
13
3
10
35
4
1
1
21
8
15
13
32
5
10
35
6
8
2
13
2
23
10
7
19
1
2
19
11
3
1
2
4
25
22
3
12
10
17
5
2
5
3
5
22
24
19
13
34
29
25
41
11
14
5
30
15
6
1
5
6
2
7
23
9
15
20
13
9
18
9
21
17
9
24
11
38
13
13
27
5
4
23
1
7
8
6
4
14
20
5
25
9
12
10
13
33
15
22
11
14
1
1
19
3
24
9
4
25
16
11
24
1
17
14
2
10
12
11
26
31
40
18
8
1
15
15
2
14
7
11
4
2
2
10
3
7
4
12
15
1
1
17
25
13
23
21
5
7
18
27
14
27
16
14
3
3
6
13
20
8
21
18
6
20
16
14
12
3
20
6
21
3
14
23
17
36
3
2
18
9
22
15
24
2
16
25
15
6
13
5
17
17
29
8
19
21
26
3
4
24
18
6
27
27
14
9
19
14
4
10
21
8
13
5
32
18
7
14
23
14
15
7
9
11
30
10
1
18
30
5
7
13
32
10
16
24
8
1
17
5
26
12
16
15
5
20
12
12
5
1
13
27
5
20
23
14
2
12
10
18
23
29
10
11
19
4
18
18
1
24
30
14
12
6
2
5
11
1
9
14
17
18
11
16
5
11
23
15
24
11
27
6
8
20
24
22
15
7
5
22
8
1
11
12
18
9
18
4
23
29
1
12
11
16
13
18
30
6
25
23
2
27
33
28
19
28
13
28
38
16
18
18
26
23
12
35
8
15
11
13
39
7
11
33
14
11
10
8
1
21
19
28
19
27
21
25
32
27
16
13
14
12
14
17


In [ ]:
dog_input[0]

In [ ]:
for i,dog in enumerate(dog_input):
    dog_outputs = output[i]
    for j,di in enumerate(dog):
        di.output = dog_outputs[j]

In [ ]:
raceDB.margin_from_dog_to_race()

In [ ]:
torch.tensor(100.1).to('cuda:0')

tensor(100.1000, device='cuda:0')

In [ ]:
torch.tensor([100.1]).to('cuda:0')

tensor([100.1000], device='cuda:0')

In [ ]:
for r in raceDB.test_race_ids:
    race = raceDB.racesDict[r]
    outputs = []
    for d in race.dogs:
        print(d)
        print(d.output)
        outputs.append(d.output)
    break
    print(output)
    race.margins = torch.stack(outputs)

tensor([100.1000], device='cuda:0')
tensor([2.4370], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([2.4468], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([0.9137], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([1.4955], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([2.2570], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([2.8256], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([2.1537], device='cuda:0', grad_fn=<SelectBackward0>)


In [ ]:
torch.cat(outputs)

tensor([100.1000,   1.2703,   1.0713,   0.9075,   1.2456,   0.7438,   0.8092,
          1.6421], device='cuda:0', grad_fn=<CatBackward0>)

TypeError: max(): argument 'input' (position 1) must be Tensor, not list

In [ ]:
output[0]

tensor([[1.0182],
        [1.1218],
        [0.7392],
        [0.6944],
        [0.6863],
        [0.7809],
        [0.6623],
        [0.2770],
        [0.4004],
        [0.3841],
        [0.4378],
        [0.6323],
        [0.6126],
        [1.2783],
        [0.4362],
        [1.2451],
        [0.7298],
        [1.1868],
        [1.6842],
        [1.3961],
        [0.9783],
        [0.3406],
        [0.6231]], device='cuda:0', grad_fn=<AddmmBackward0>)

RuntimeError: stack expects each tensor to be equal size, but got [] at entry 0 and [1] at entry 1

In [ ]:
test_races = [raceDB.racesDict[x] for x in raceDB.test_race_ids][0]

In [ ]:
test

NameError: name 'test' is not defined

In [ ]:
print(test_races.margins)

tensor([100.1000,   2.4370,   2.4468,   0.9137,   1.4955,   2.2570,   2.8256,
          2.1537], device='cuda:0', grad_fn=<CatBackward0>)


In [ ]:
[x.dog.dog_name for x in test_races.dogs]

['no_name',
 'MR. MUGGINS',
 'RETREAD PETE',
 "HOW'S THE LUCK",
 'CHLOE WHO',
 'HOMEBUSH THAW',
 'HOMEBUSH LEROY',
 'GOLDSTAR HOPE']

In [ ]:
sweep_config = {"method": "random"}

metric = {"name": "ROI < 30", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "f1_layer_size": {"values": [256]},
    "f2_layer_size": {"values": [64]},
    "dropout": {"values": [0.3]},
    "len_data": {"value": len(raceDB.raceIDs)},
    "hidden_size": {"value":64}
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [100]},
        "validation_split": {"value": 0.1},
        "loss": {
            "values": [ "CEL"],
            # "values": ["Huber", "MSE", "L1", "BCE", "Custom", "KL"]
            # 'value': 'l1_custom'
        },
        "num_layers": {"values": [2]},
    }
)

parameters_dict.update(
    {
        "learning_rate": {
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0.00001,
            "max": 0.001,
        },
        "l1_beta": {"value": 0.1
        },
        "batch_size": {
            'values': [50,125,250,500]
        },
        "batch_before_backwards": {
            'values': [5,10,15,50]
        }
    }
)

import pprint

pprint.pprint(sweep_config)


sweep_config

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'ROI < 30'},
 'parameters': {'batch_before_backwards': {'values': [5, 10, 15, 50]},
                'batch_size': {'values': [50, 125, 250, 500]},
                'dropout': {'values': [0.3]},
                'epochs': {'values': [100]},
                'f1_layer_size': {'values': [256]},
                'f2_layer_size': {'values': [64]},
                'hidden_size': {'value': 64},
                'l1_beta': {'value': 0.1},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.001,
                                  'min': 1e-05},
                'len_data': {'value': 88865},
                'loss': {'values': ['CEL']},
                'num_layers': {'values': [2]},
                'optimizer': {'value': 'adamW'},
                'validation_split': {'value': 0.1}}}


{'method': 'random',
 'metric': {'name': 'ROI < 30', 'goal': 'maximize'},
 'parameters': {'optimizer': {'value': 'adamW'},
  'f1_layer_size': {'values': [256]},
  'f2_layer_size': {'values': [64]},
  'dropout': {'values': [0.3]},
  'len_data': {'value': 88865},
  'hidden_size': {'value': 64},
  'epochs': {'values': [100]},
  'validation_split': {'value': 0.1},
  'loss': {'values': ['CEL']},
  'num_layers': {'values': [2]},
  'learning_rate': {'distribution': 'uniform', 'min': 1e-05, 'max': 0.001},
  'l1_beta': {'value': 0.1},
  'batch_size': {'values': [50, 125, 250, 500]},
  'batch_before_backwards': {'values': [5, 10, 15, 50]}}}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="LSTM_sweeps")
CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)

Create sweep with ID: udnlmppo
Sweep URL: https://wandb.ai/nickojelly/LSTM_sweeps/sweeps/udnlmppo


wandb: Agent Starting Run: ctlxz40f with config:
wandb: 	batch_before_backwards: 10
wandb: 	batch_size: 125
wandb: 	dropout: 0.3
wandb: 	epochs: 100
wandb: 	f1_layer_size: 256
wandb: 	f2_layer_size: 64
wandb: 	hidden_size: 64
wandb: 	l1_beta: 0.1
wandb: 	learning_rate: 0.0004436610065404108
wandb: 	len_data: 88865
wandb: 	loss: CEL
wandb: 	num_layers: 2
wandb: 	optimizer: adamW
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'batch_before_backwards': 10, 'batch_size': 125, 'dropout': 0.3, 'epochs': 100, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 64, 'l1_beta': 0.1, 'learning_rate': 0.0004436610065404108, 'len_data': 88865, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
100
{'batch_before_backwards': 10, 'batch_size': 125, 'dropout': 0.3, 'epochs': 100, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 64, 'l1_beta': 0.1, 'learning_rate': 0.0004436610065404108, 'len_data': 88865, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
smalll_lin_GRUNet(
  (batchnorm): BatchNorm1d(510, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc0): Linear(in_features=510, out_features=510, bias=True)
  (gru1): GRUCell(510, 64)
  (gru2): GRUCell(510, 64)
  (gru3): GRUCell(510, 64)
  (gru4): GRUCell(510, 64)
  (gru5): GRUCell(510, 64)
  (gru6): GRUCell(510, 64)
  (gru7): GRUCell(510, 64)
  (gru8): GRUCell(510, 64)
  (rl1): 

  0%|          | 0/100 [00:00<?, ?it/s]


Run ctlxz40f errored: RuntimeError('hidden0 has inconsistent hidden_size: got 128, expected 64')
wandb: ERROR Run ctlxz40f errored: RuntimeError('hidden0 has inconsistent hidden_size: got 128, expected 64')
wandb: Agent Starting Run: 3ky1vnz4 with config:
wandb: 	batch_before_backwards: 15
wandb: 	batch_size: 125
wandb: 	dropout: 0.3
wandb: 	epochs: 100
wandb: 	f1_layer_size: 256
wandb: 	f2_layer_size: 64
wandb: 	hidden_size: 64
wandb: 	l1_beta: 0.1
wandb: 	learning_rate: 3.554693043910414e-05
wandb: 	len_data: 88865
wandb: 	loss: CEL
wandb: 	num_layers: 2
wandb: 	optimizer: adamW
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

  0%|          | 0/100 [00:00<?, ?it/s]


Run 3ky1vnz4 errored: RuntimeError('hidden0 has inconsistent hidden_size: got 128, expected 64')
wandb: ERROR Run 3ky1vnz4 errored: RuntimeError('hidden0 has inconsistent hidden_size: got 128, expected 64')
wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: hf60k4q6 with config:
wandb: 	batch_before_backwards: 50
wandb: 	batch_size: 500
wandb: 	dropout: 0.3
wandb: 	epochs: 100
wandb: 	f1_layer_size: 256
wandb: 	f2_layer_size: 64
wandb: 	hidden_size: 64
wandb: 	l1_beta: 0.1
wandb: 	learning_rate: 0.0004939104475117235
wandb: 	len_data: 88865
wandb: 	loss: CEL
wandb: 	num_layers: 2
wandb: 	optimizer: adamW
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'batch_before_backwards': 50, 'batch_size': 500, 'dropout': 0.3, 'epochs': 100, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 64, 'l1_beta': 0.1, 'learning_rate': 0.0004939104475117235, 'len_data': 88865, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
100
{'batch_before_backwards': 50, 'batch_size': 500, 'dropout': 0.3, 'epochs': 100, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 64, 'l1_beta': 0.1, 'learning_rate': 0.0004939104475117235, 'len_data': 88865, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
smalll_lin_GRUNet(
  (batchnorm): BatchNorm1d(510, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc0): Linear(in_features=510, out_features=510, bias=True)
  (gru1): GRUCell(510, 64)
  (gru2): GRUCell(510, 64)
  (gru3): GRUCell(510, 64)
  (gru4): GRUCell(510, 64)
  (gru5): GRUCell(510, 64)
  (gru6): GRUCell(510, 64)
  (gru7): GRUCell(510, 64)
  (gru8): GRUCell(510, 64)
  (rl1): 

  0%|          | 0/100 [00:00<?, ?it/s]


Run hf60k4q6 errored: RuntimeError('hidden0 has inconsistent hidden_size: got 128, expected 64')
wandb: ERROR Run hf60k4q6 errored: RuntimeError('hidden0 has inconsistent hidden_size: got 128, expected 64')
Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true


In [ ]:

stop
model = GRUNet(203,64).to(device)
model.load_state_dict(model_state_dict)

NameError: name 'stop' is not defined

In [ ]:

dataset.create_hidden_states_dict()